# ERA5 - download data
From this [website](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview). Follow their instructions on how to [install their CDS API](https://cds.climate.copernicus.eu/api-how-to) and then copy/paste the Python "MARSREQUEST" code into this notebook to download the relevant variables for your budget in netcdf format.

## ORAS5: 
> https://www.ecmwf.int/en/research/climate-reanalysis/ocean-reanalysis

> https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-oras5?tab=overview

In [1]:
import cdsapi

c = cdsapi.Client()

In [2]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-oras5',
    {
        'format': 'zip',
        'vertical_resolution': 'all_levels',
        'variable': 'potential_temperature',
        'year': '2023',
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10',
        ],
        'product_type': 'operational',
    },
    'pot-temp.zip')

2023-11-20 15:38:22,762 INFO Welcome to the CDS
2023-11-20 15:38:22,762 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-oras5
2023-11-20 15:38:22,987 INFO Request is queued
2023-11-20 15:38:24,116 INFO Request is running
2023-11-20 15:42:42,614 INFO Request is completed
2023-11-20 15:42:42,615 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data2/dataset-reanalysis-oras5-89427c8b-6de0-456d-8b1f-8394a8f83cbf.zip to pot-temp.zip (1.7G)
2023-11-20 15:45:20,732 INFO Download rate 11.3M/s    


Result(content_length=1874123662,content_type=application/zip,location=https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data2/dataset-reanalysis-oras5-89427c8b-6de0-456d-8b1f-8394a8f83cbf.zip)

In [3]:
pwd

'/home/jovyan/MHW-WMT'

In [4]:
!unzip '/home/jovyan/MHW-WMT/pot-temp.zip'

Archive:  /home/jovyan/MHW-WMT/pot-temp.zip
  inflating: votemper_control_monthly_highres_3D_202308_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202305_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202309_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202303_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202302_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202310_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202304_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202307_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202306_OPER_v0.1.nc  
  inflating: votemper_control_monthly_highres_3D_202301_OPER_v0.1.nc  


In [3]:
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            'total_precipitation', 
        ],
        'year': ['2023'],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'format': 'netcdf',
    },
    'precip-2023.nc')

2023-11-17 17:21:11,212 INFO Welcome to the CDS
2023-11-17 17:21:11,212 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-11-17 17:21:11,419 INFO Request is queued
2023-11-17 17:21:12,550 INFO Request is running
2023-11-17 17:37:35,453 INFO Request is completed
2023-11-17 17:37:35,453 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data1/adaptor.mars.internal-1700242110.04108-5839-7-57106833-2043-40f8-8b9a-4d4248a326f7.nc to precip-2023.nc (29.3G)
2023-11-17 18:47:10,302 INFO Download rate 7.2M/s       


Result(content_length=31471171008,content_type=application/x-netcdf,location=https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data1/adaptor.mars.internal-1700242110.04108-5839-7-57106833-2043-40f8-8b9a-4d4248a326f7.nc)

In [3]:
var_list = ['runoff', 'surface_runoff', 'sea_surface_temperature', 'surface_latent_heat_flux', 
            'surface_net_solar_radiation', 'surface_net_thermal_radiation', 'surface_sensible_heat_flux', 
            'e',]
#['sub_surface_runoff', 'surface_solar_radiation_downwards', 'surface_thermal_radiation_downwards',]

for i in var_list:
    print(f'starting {i} download')
    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'variable': [
                i, 
            ],
            'year': ['2022'],
            'month': [
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
            'format': 'netcdf',
        },
        f'2022_winter_{i}.nc')

starting runoff download


2023-10-11 10:54:01,623 INFO Welcome to the CDS
2023-10-11 10:54:01,624 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-10-11 10:54:01,923 INFO Request is queued
2023-10-11 11:02:22,907 INFO Request is running
2023-10-11 11:06:24,442 INFO Request is completed
2023-10-11 11:06:24,443 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data6/adaptor.mars.internal-1697022201.721435-31617-16-073c0b92-f60f-47a6-bdaf-ec4530043e11.nc to 2022_winter_runoff.nc (4.3G)
2023-10-11 11:15:18,653 INFO Download rate 8.2M/s      
2023-10-11 11:15:19,382 INFO Welcome to the CDS
2023-10-11 11:15:19,383 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


starting surface_runoff download


2023-10-11 11:15:19,556 INFO Request is queued
2023-10-11 11:23:40,594 INFO Request is running
2023-10-11 11:29:42,526 INFO Request is completed
2023-10-11 11:29:42,527 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data3/adaptor.mars.internal-1697023500.7263615-16061-7-643dfd50-c5f5-4d23-867f-b7d54b242596.nc to 2022_winter_surface_runoff.nc (4.3G)
2023-10-11 12:07:14,081 INFO Download rate 1.9M/s      
2023-10-11 12:07:14,799 INFO Welcome to the CDS
2023-10-11 12:07:14,800 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


starting sea_surface_temperature download


...

2023-10-11 12:48:29,665 INFO Request is queued
2023-10-11 12:54:50,007 INFO Request is running
2023-10-11 12:58:51,226 INFO Request is completed
2023-10-11 12:58:51,226 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data3/adaptor.mars.internal-1697028920.8168418-24110-6-dd5d126e-976f-4910-a4bd-fa57ad3b4ee1.nc to 2022_winter_surface_net_thermal_radiation.nc (4.3G)
2023-10-11 13:07:24,985 INFO Download rate 8.5M/s      
2023-10-11 13:07:25,854 INFO Welcome to the CDS
2023-10-11 13:07:25,855 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


starting surface_sensible_heat_flux download


2023-10-11 13:07:26,016 INFO Request is queued
2023-10-11 13:13:46,408 INFO Request is running
2023-10-11 13:19:48,336 INFO Request is completed
2023-10-11 13:19:48,337 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data6/adaptor.mars.internal-1697030178.0263522-13781-6-ab3895cf-68ae-41d1-b01b-db0215e5504a.nc to 2022_winter_surface_sensible_heat_flux.nc (4.3G)
2023-10-11 13:23:51,697 INFO Download rate 18M/s       
2023-10-11 13:23:52,414 INFO Welcome to the CDS
2023-10-11 13:23:52,414 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


starting e download


2023-10-11 13:23:52,616 INFO Request is queued
2023-10-11 13:30:13,045 INFO Request is running
2023-10-11 13:34:14,409 INFO Request is completed
2023-10-11 13:34:14,410 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data5/adaptor.mars.internal-1697031081.1520853-19406-5-325333a1-96dd-4e1a-bcde-fe2a3276c8e8.nc to 2022_winter_e.nc (4.3G)
2023-10-11 14:17:26,384 INFO Download rate 1.7M/s      


In [ ]:
'runoff', 'sub_surface_runoff', 'surface_runoff', 'sea_surface_temperature', 'surface_latent_heat_flux', 
'surface_net_solar_radiation', 'surface_net_thermal_radiation', 'surface_sensible_heat_flux', 
'surface_solar_radiation_downwards', 'surface_thermal_radiation_downwards',